In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/RuSportSum/ods_broadcasts_201905301157.csv.zip" /content/ods_broadcasts_201905301157.csv.zip
!unzip /content/ods_broadcasts_201905301157.csv.zip

!cp "/content/drive/MyDrive/RuSportSum/result_data.csv" /content/result_data.csv

Archive:  /content/ods_broadcasts_201905301157.csv.zip
  inflating: ods_broadcasts_201905301157.csv  


In [ ]:
!pip install -U --quiet sentence-transformers evaluate nltk rouge_score bert_score razdel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import re
import numpy as np
import nltk
import datetime as dt
import random as ra
import evaluate

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from razdel import sentenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
data_broadcasts = pd.read_csv("/content/ods_broadcasts_201905301157.csv")

def remove_tags(html):
    soup = BeautifulSoup(str(html), "html.parser")

    for data in soup(['style', 'script']):
        data.decompose()

    return ' '.join(soup.stripped_strings)

data_broadcasts['content'] = data_broadcasts['content'].apply(remove_tags)

data_broadcasts['content'] = data_broadcasts['minute'].apply(lambda x: str(x)) + " минута. " + data_broadcasts['content']
data_broadcasts['content'] = data_broadcasts['content'].apply(lambda x: re.sub(r'[гГ]+[оО]+[лЛ]+', 'гол', x))

data_broadcasts['match_time'] = data_broadcasts['match_time'].apply(lambda x: dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
data_broadcasts['match_time'] = pd.to_datetime(data_broadcasts['match_time'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

data_broadcasts = data_broadcasts.sort_values(['match_id', 'minute', 'message_time'], ascending = True)
data_broadcasts

<ipython-input-5-b9eaafd1f952>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")
<ipython-input-5-b9eaafd1f952>:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(str(html), "html.parser")


,match_id,team1,team2,name,match_time,type,minute,content,message_time
0,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,0 минута. Добрый день! Sports.ru поздравляет в...,1237034763
1,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. Будем надеяться, что признаков пресл...",1237034996
2,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. Что ж, сегодняшних соперников можно ...",1237035202
3,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,0 минута. Зимой подмосковный клуб едва не исче...,1237035415
4,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. ""Локомотив"" зимой старался стать луч...",1237035796
...,...,...,...,...,...,...,...,...,...
722062,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,empty,63,"63 минута. 2:1 - ""Питтсбург"" выигрывает второй...",1464837758
722063,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,empty,63,"63 минута. Очевидно, что залог успеха ""Питтсбу...",1464837879
722064,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,empty,63,"63 минута. Другими словами, ДеБур должен в пер...",1464838029
722065,1104866,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-06-02 00:00:00,empty,63,63 минута. Большое спасибо за чтение! До новых...,1464838220


In [ ]:
data_broadcasts.type.value_counts()

empty         365009
attention      72590
zamena         31702
yellowcard     22010
ball           17484
puck           11308
penalty         8131
tenball         5690
ace             2043
break           1497
redcard         1422
double          1372
2points          755
gk_zamena        523
fight            465
3points          406
freethrow2       226
freethrow1       153
timeout          122
injury            19
delete             3
Name: type, dtype: int64

In [ ]:
content_type = [
    "empty",
    "attention",
    "zamena",
    "yellowcard",
    "ball",
    "puck",
    "penalty",
    "tenball",
    "ace",
    "break",
    "redcard",
    "double",
    "2points",
    "gk_zamena",
    "fight",
    "3points",
    "freethrow2",
    "freethrow1",
    "timeout",
    "injury",
    "delete"
    ]

In [ ]:
result_data = pd.read_csv('/content/result_data.csv')
result_data

,match_id,team1,team2,broadcast_name,match_time,concatenate_content,last_part_content,id,news_name,ctime,body,name_body,len_name_body,result_match
0,455452,Дания,Германия,Чемпионат Европы,1339958700,Всем привет и добро пожаловать на ЕВРО-2012 бе...,Бендтнер зарабатывал этот угловой с таким осте...,141499717,Ларс Бендер забил первый гол за сборную Германии,2012-06-18 00:50:25,Полузащитник «Байера» и сборной Германии Ларс ...,Ларс Бендер забил первый гол за сборную Герман...,79,1
1,476651,Бавария,Челси,Лига чемпионов,1337453100,Привет-привет! Настраиваемся? http://www.sport...,"Бить будут в ворота, за которыми мюнхенский су...",140901107,Дрогба сравнял счет в матче с «Баварией»,2012-05-20 00:34:22,Форвард «Челси» Дидье Дрогба забил гол в финал...,Дрогба сравнял счет в матче с «Баварией». Форв...,43,0
2,481837,Кубань,Спартак,Россия. Премьер-лига,1347714000,"Здравствуйте, друзья. Продолжаем следить за со...","А меняет Прудников Кулика. ""Спартак"" ответил з...",143405274,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,2012-09-15 12:57:47,Сегодня в Краснодаре в рамках 8-го тура РФПЛ м...,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,94,1
3,1089413,СКА,Локомотив,Кубок Гагарина,1456417800,"Привет всем, кто не представляет своей жизни б...",3:0 и 2-1 в серии - СКА одержал пока что свою ...,1037500549,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",2016-02-25 22:21:05,Голкипер СКА Микко Коскинен сделал шатаут в тр...,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",43,1
4,1092758,СКА,Динамо Москва,Кубок Гагарина,1457877600,"Привет всем, кто не представляет своей жизни б...",Момент у СКА! Кетов после рывка Каблукова брос...,1038079919,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,2016-03-13 20:14:38,СКА обыграл московское «Динамо» (4:0) в четвер...,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,55,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12833,815435,СКА,Авангард,КХЛ,1393428600,"Привет всем, кто не представляет своей жизни б...",Отмечу прекрасную статистику Ружички после пер...,158241356,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,2014-02-26 19:31:03,Сегодня в Петербурге проходит матч регулярного...,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,31,1
12834,1013451,Барыс,Авангард,Кубок Гагарина,1425389400,Всем привет! Сегодня в Кубке Гагарина большой ...,"Хозяева, как и во втором матче, переводят игру...",1027691296,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,2015-03-04 14:39:39,Сегодня в четвертой игре серии первого раунда ...,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,40,1
12835,799065,Атлетик,Барселона,Испания. Ла Лига,1385928000,"Салют, амигос! ""Атлетик"" - ""Барселона"" - проти...",Чуть было сами себе не привезли хозяева! Оступ...,154776181,«Атлетик» – «Барселона». Пике проводит 150-й м...,2013-12-02 00:24:55,Защитник «Барселоны» Жерар Пике вышел в старто...,«Атлетик» – «Барселона». Пике проводит 150-й м...,71,0
12836,786385,Манчестер Юнайтед,Халл,Англия. Премьер-лига,1399401900,"В последних трех матчах АПЛ ""Халл"" набрал лишь...","Феллайни совсем не в склад, не в лад. Кагава е...",1020359093,«Манчестер Юнайтед» – «Халл». Джонс был госпит...,2014-05-07 00:23:57,"Защитник «Манчестер Юнайтед» Фил Джонс, получи...",«Манчестер Юнайтед» – «Халл». Джонс был госпит...,79,0


In [ ]:
# Метрика
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

In [ ]:
id = 481837
temp_news = result_data.iloc[2]['name_body']
sentences = list(sentenize(temp_news))

print(sentences)

predictions = data_broadcasts[data_broadcasts['match_id'] == id]['content'].to_list()

for sentence in sentences:
  references = [sentence.text for _ in range(len(predictions))]
  bertscore_results = bertscore.compute(predictions=predictions, references=references, lang="ru")
  rouge_results = rouge.compute(predictions=predictions, references=references, tokenizer=lambda x: x.split(), use_aggregator=False)

  bertscores = [round(v, 3) for v in bertscore_results["f1"]]
  rouge1s = [round(v, 3) for v in rouge_results["rouge1"]]
  rouge2s = [round(v, 3) for v in rouge_results["rouge2"]]
  scores = []
  for i in range(len(bertscores)):
    scores.append(0.7 * bertscores[i] + 0.3 * (rouge1s[i] + 2 * rouge2s[i]))

  max_score = max(scores)
  max_index = scores.index(max_score)

  print(predictions[max_index])

[Substring(0, 27, '«Кубань» – «Спартак» – 2:2.'), Substring(28, 57, 'Озбилиз сравнял счет в матче.'), Substring(58, 175, 'Сегодня в Краснодаре в рамках 8-го тура РФПЛ местная «Кубань» принимала на одноименном стадионе московский «Спартак».'), Substring(176, 239, 'Sports.ru провел онлайн-трансляцию матча «Кубань» – «Спартак» .'), Substring(240, 269, 'Матч завершился вничью – 2:2.'), Substring(270, 420, 'На 14-й минуте счет открыл Эйден Макгиди после передачи Артема Дзюбы, на 27-й минуте дальним ударом из-за пределов штрафной счет сравнял Артем Фидлер.'), Substring(421, 591, 'Во втором тайме на 56-й минуте «красно-белых» вывел вперед вышедший на замену в перерыве Эммануэль Эменике, однако через 7 минут равенство опять восстановил Араз Озбилиз.'), Substring(592, 664, 'Таблица российской премьер-лиги Календарь матчей российской премьер-лиги')]
93 минута. Финальный свисток. Матч завершается вничью - 2:2.
62 минута. Озбилиз заработал стандарт в довольно опасной зоне. Посмотрим.
0 минута. Здр

In [ ]:
references[0].text

'Дрогба сравнял счет в матче с «Баварией».'

In [ ]:
rouge_results = rouge.compute(predictions=predictions, references=references, tokenizer=lambda x: x.split(), use_aggregator=False)

In [ ]:
data_contents = data_broadcasts[data_broadcasts['match_id'] == 476651]
data_contents['bertscore_f1'] = [round(v, 3) for v in bertscore_results["f1"]]
data_contents['rouge1'] = [round(v, 3) for v in rouge_results["rouge1"]]
data_contents['rouge2'] = [round(v, 3) for v in rouge_results["rouge2"]]
data_contents['score'] = 0.7 * data_contents['bertscore_f1'] + (1 - 0.7) * (data_contents['rouge1'] + 2 * data_contents['rouge2'])

<ipython-input-58-0a34fb870ad7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_contents['bertscore_f1'] = [round(v, 3) for v in bertscore_results["f1"]]
<ipython-input-58-0a34fb870ad7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_contents['rouge1'] = [round(v, 3) for v in rouge_results["rouge1"]]
<ipython-input-58-0a34fb870ad7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [ ]:
temp_news

'Дрогба сравнял счет в матче с «Баварией». Форвард «Челси» Дидье Дрогба забил гол в финале Лиги чемпионов против «Баварии». На 88-й минуте встречи после подачи хавбека «Челси» Хуана Маты с углового Дрогба переправил головой мяч в ворота Мануэля Нойера. «Бавария» – «Челси». Онлайн'

In [ ]:
data_contents

,match_id,team1,team2,name,match_time,type,minute,content,message_time,bertscore_f1,rouge1,rouge2,score
265472,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,empty,0,0 минута. Привет-привет! Настраиваемся? http:/...,1337447226,0.548,0.000,0.0,0.4932
265473,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,empty,0,0 минута. Сейчас самый волнительный момент - п...,1337451614,0.576,0.015,0.0,0.5229
265474,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,empty,0,"0 минута. Тем более, что поводов для оной пред...",1337451750,0.644,0.115,0.0,0.6141
265475,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,empty,0,0 минута. Для Хайнкеса это третий финал евроку...,1337451876,0.647,0.074,0.0,0.6045
265476,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,empty,0,"0 минута. Что до ""Челси"", то лондонский клуб в...",1337452356,0.642,0.000,0.0,0.5778
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265582,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,attention,125,125 минута. Швайни на пятый удар у немцев. ШТА...,1337462941,0.599,0.000,0.0,0.5391
265583,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,ball,125,125 минута. Дрогба - выигрывать титул? ДА!!! г...,1337463116,0.593,0.036,0.0,0.5445
265584,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,empty,125,125 минута. Выход с первого места в группе при...,1337463606,0.647,0.093,0.0,0.6102
265585,476651,Бавария,Челси,Лига чемпионов,2012-05-19 18:45:00,empty,125,"125 минута. Дрогба, Чех, Лэмпард... И Терри с ...",1337463794,0.618,0.068,0.0,0.5766


## Обобщающие новости

In [ ]:
result_match_data = result_data[result_data['result_match'] == 1]

relevant_match_ids = result_match_data['match_id'].drop_duplicates().tolist()

relevant_data_broadcasts = data_broadcasts[data_broadcasts['match_id'].isin(relevant_match_ids)].reset_index(drop=True)
relevant_data_news = result_match_data

In [ ]:
count_news = len(relevant_data_news)
print("Количество новостей:", count_news)

Количество новостей: 8749


#### Только реплики, которые имеют непустой тип (атрибут type)

In [ ]:
df = {}
count_df_row = 0

for index in range(len(relevant_data_news)):
  current_news = relevant_data_news.iloc[index]

  # все важные реплики из трансляции
  important_replicas = relevant_data_broadcasts[(relevant_data_broadcasts['match_id'] == current_news['match_id']) \
                                                & (relevant_data_broadcasts['type'].isin(content_type[2:]))]
  join_important_replicas = ' '.join(important_replicas.content.tolist())

  df['row_' + str(count_df_row)] = [
      current_news['match_id'],
      current_news['id'],
      current_news['broadcast_name'],
      current_news['team1'],
      current_news['team2'],
      join_important_replicas,
      current_news['news_name'],
      current_news['body']
      ]
  count_df_row += 1

In [ ]:
dt_frame = pd.DataFrame.from_dict(df, orient='index', columns=['match_id', 'news_id', 'name', 'team1', 'team2', 'content', 'news_name', 'news_body']).reset_index(drop=True)
dt_frame

,match_id,news_id,name,team1,team2,content,news_name,news_body
0,455452,141499717,Чемпионат Европы,Дания,Германия,19 минута. гол! Не Гомесом единым! Подольски п...,Ларс Бендер забил первый гол за сборную Германии,Полузащитник «Байера» и сборной Германии Ларс ...
1,481837,143405274,Россия. Премьер-лига,Кубань,Спартак,15 минута. гол! Макгиди прибежал в штрафную на...,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,Сегодня в Краснодаре в рамках 8-го тура РФПЛ м...
2,1089413,1037500549,Кубок Гагарина,СКА,Локомотив,15 минута. гол!!!! Широков - 1:0! 2-й гол в пл...,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",Голкипер СКА Микко Коскинен сделал шатаут в тр...
3,1092758,1038079919,Кубок Гагарина,СКА,Динамо Москва,3 минута. гол!!!! Шипачев - 1:0! 5-й гол в пле...,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,СКА обыграл московское «Динамо» (4:0) в четвер...
4,787214,1020441523,Россия. Премьер-лига,Ахмат,Волга НН,32 минута. Первое предупреждение не заставило ...,«Волга» вылетела в ФНЛ,Уступив в матче 29-го тура чемпионата России п...
...,...,...,...,...,...,...,...,...
8744,963066,1029756828,Россия. Премьер-лига,Арсенал Тула,Ахмат,23 минута. гол! 0:1! голеада Мбенгуе продолжае...,«Арсенал» Тула – «Терек» – 1:1. Команды разошл...,В 28-м туре чемпионата России «Арсенал» разоше...
8745,478494,141113093,Кубок Стэнли,Нью-Джерси,Лос-Анджелес,10 минута. ФРЭЗЕР - 0:1!!! 1-й гол в плей-офф!...,НХЛ. «Лос-Анджелес» одержал девятую подряд вые...,"«Лос-Анджелес», обыграв «Нью-Джерси» (2:1 ОТ) ..."
8746,815435,158241356,КХЛ,СКА,Авангард,1 минута. СЕРГЕЙ КОСТИЦЫН - 0:1!!! 10-й гол в ...,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,Сегодня в Петербурге проходит матч регулярного...
8747,1013451,1027691296,Кубок Гагарина,Барыс,Авангард,7 минута. гол! 1:0! Руденко! Получил шайбу в у...,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,Сегодня в четвертой игре серии первого раунда ...


In [ ]:
dt_frame['news_name_body'] = dt_frame['news_name'] + ' & ' + dt_frame['news_body']

In [ ]:
train_news_am, eval_news_am = train_test_split(dt_frame.sample(frac=1, random_state=42), test_size=1_000, random_state=42)

train_news_am, test_news_am = train_test_split(train_news_am, test_size=150, random_state=42)

In [ ]:
train_news_am

,match_id,news_id,name,team1,team2,content,news_name,news_body,news_name_body
1734,965329,1029966958,Германия. Бундеслига,Бавария,Майнц,25 минута. гол! Левандовски открывает счет! Бь...,«Бавария» – «Майнц» – 2:0. Швайнштайгер постав...,В 34-м туре бундеслиги «Бавария» обыграла «Май...,«Бавария» – «Майнц» – 2:0. Швайнштайгер постав...
2877,907948,159287107,Кубок Гагарина,Локомотив,Лев,25 минута. НОВОТНЫ - 0:1!!! 4-й гол в плей-офф...,КХЛ. Кубок Гагарина. 1/2 финала. Четвертый мат...,Сегодня в четвертом матче финала Западной конф...,КХЛ. Кубок Гагарина. 1/2 финала. Четвертый мат...
7364,965186,1026902126,Германия. Бундеслига,Байер,Боруссия Д,13 минута. Дрмич получил первое предупреждение...,«Байер» – «Боруссия» Дортмунд – 0:0. Команды г...,Сегодня в рамках 18-го тура чемпионата Германи...,«Байер» – «Боруссия» Дортмунд – 0:0. Команды г...
398,1035457,1040243081,Россия. Премьер-лига,Ахмат,Ростов,24 минута. А вот и анонсированная замена. Вмес...,«Терек» – «Ростов» – 0:2. Ерохин установил око...,В 30-м туре премьер-лиги «Терек» уступил «Рост...,«Терек» – «Ростов» – 0:2. Ерохин установил око...
6765,490290,143886070,Франция. Лига 1,Марсель,ПСЖ,"17 минута. гол! 1:0! Угловой у ворот хозяев, н...",«Марсель» – «ПСЖ» – 2:2. Жиньяк сравнял счет,Лидер французского чемпионата «Марсель» на «Ве...,«Марсель» – «ПСЖ» – 2:2. Жиньяк сравнял счет &...
...,...,...,...,...,...,...,...,...,...
207,829996,1021343013,Чемпионат мира,Англия,Италия,35 минута. гол! 0:1! Маркизио! Разыграли углов...,Англия – Италия – 1:2. Балотелли забил победны...,Сегодня в 1-м туре чемпионата мира-2014 на ста...,Англия – Италия – 1:2. Балотелли забил победны...
7852,830027,1021785801,Чемпионат мира,Голландия,Мексика,9 минута. Ничего себе начало! Мы только начали...,Голландия – Мексика. Голландцы впервые за 26 м...,Сборная Голландии сегодня обыграла команду Мек...,Голландия – Мексика. Голландцы впервые за 26 м...
6535,775308,147870527,Кубок Гагарина,Трактор,Ак Барс,4 минута. НИЧУШКИН - 1:0!!!!! 6-й гол в плей-о...,КХЛ. Кубок Гагарина. 1/2 финала. «Трактор» в о...,Сегодня в четвертом матче полуфинальной серии ...,КХЛ. Кубок Гагарина. 1/2 финала. «Трактор» в о...
4128,814916,153516257,КХЛ,Барыс,Динамо Москва,22 минута. ЦВЕТКОВ - 0:1!!! 4-й гол в сезоне! ...,«Барыс» – «Динамо» – 4:3 Б. Хозяева одержали в...,Сегодня в рамках очередного матча регулярного ...,«Барыс» – «Динамо» – 4:3 Б. Хозяева одержали в...


In [ ]:
train_news_am.to_csv('train_after_match_pairs_data1.csv', index=False, encoding='utf-8')
eval_news_am.to_csv('eval_after_match_pairs_data1.csv', index=False, encoding='utf-8')
test_news_am.to_csv('test_after_match_pairs_data1.csv', index=False, encoding='utf-8')

!cp /content/train_after_match_pairs_data1.csv "/content/drive/MyDrive/RuSportSum/train_after_match_pairs_data1.csv"
!cp /content/eval_after_match_pairs_data1.csv "/content/drive/MyDrive/RuSportSum/eval_after_match_pairs_data1.csv"
!cp /content/test_after_match_pairs_data1.csv "/content/drive/MyDrive/RuSportSum/test_after_match_pairs_data1.csv"

In [ ]:
# данные для сверки, что везде одинаковые наборы
train_data = train_news_am
eval_data = eval_news_am
test_data = test_news_am

#### Реплики, которые имеют непустой тип; концовка

In [ ]:
df = {}
count_df_row = 0

for index in range(len(relevant_data_news)):
  current_news = relevant_data_news.iloc[index]

  # все важные реплики из трансляции
  important_replicas = relevant_data_broadcasts[(relevant_data_broadcasts['match_id'] == current_news['match_id']) \
                                                & (relevant_data_broadcasts['type'].isin(content_type[2:]))]
  join_important_replicas = ' '.join(important_replicas.content.tolist())

  # концовка матча (все реплики, после реплики с каким-то типом)
  match_replicas = data_broadcasts[data_broadcasts['match_id'] == current_news['match_id']].reset_index()
  mask = match_replicas['type'].isin(content_type[1:]).to_list()
  try:
    index_true = mask[::-1].index(True)
  except:
    index_true = 1 if len(mask) > 1 else 0
  for i in range(len(mask) - index_true):
    mask[i] = True
  end_replicas = match_replicas[~pd.Series(mask)]
  join_end_match = ' '.join(end_replicas.content.tolist())

  df['row_' + str(count_df_row)] = [
      current_news['match_id'],
      current_news['id'],
      current_news['broadcast_name'],
      current_news['team1'],
      current_news['team2'],
      join_important_replicas + ' ' + join_end_match,
      current_news['news_name'],
      current_news['body']
      ]
  count_df_row += 1

In [ ]:
dt_frame = pd.DataFrame.from_dict(df, orient='index', columns=['match_id', 'news_id', 'name', 'team1', 'team2', 'content', 'news_name', 'news_body']).reset_index(drop=True)
dt_frame

,match_id,news_id,name,team1,team2,content,news_name,news_body
0,455452,141499717,Чемпионат Европы,Дания,Германия,19 минута. гол! Не Гомесом единым! Подольски п...,Ларс Бендер забил первый гол за сборную Германии,Полузащитник «Байера» и сборной Германии Ларс ...
1,481837,143405274,Россия. Премьер-лига,Кубань,Спартак,15 минута. гол! Макгиди прибежал в штрафную на...,«Кубань» – «Спартак» – 2:2. Озбилиз сравнял сч...,Сегодня в Краснодаре в рамках 8-го тура РФПЛ м...
2,1089413,1037500549,Кубок Гагарина,СКА,Локомотив,15 минута. гол!!!! Широков - 1:0! 2-й гол в пл...,"СКА – «Локомотив». Коскинен сыграл на ноль, от...",Голкипер СКА Микко Коскинен сделал шатаут в тр...
3,1092758,1038079919,Кубок Гагарина,СКА,Динамо Москва,3 минута. гол!!!! Шипачев - 1:0! 5-й гол в пле...,СКА выиграл 5 матчей из 5 в плей-офф без Ковал...,СКА обыграл московское «Динамо» (4:0) в четвер...
4,787214,1020441523,Россия. Премьер-лига,Ахмат,Волга НН,32 минута. Первое предупреждение не заставило ...,«Волга» вылетела в ФНЛ,Уступив в матче 29-го тура чемпионата России п...
...,...,...,...,...,...,...,...,...
8744,963066,1029756828,Россия. Премьер-лига,Арсенал Тула,Ахмат,23 минута. гол! 0:1! голеада Мбенгуе продолжае...,«Арсенал» Тула – «Терек» – 1:1. Команды разошл...,В 28-м туре чемпионата России «Арсенал» разоше...
8745,478494,141113093,Кубок Стэнли,Нью-Джерси,Лос-Анджелес,10 минута. ФРЭЗЕР - 0:1!!! 1-й гол в плей-офф!...,НХЛ. «Лос-Анджелес» одержал девятую подряд вые...,"«Лос-Анджелес», обыграв «Нью-Джерси» (2:1 ОТ) ..."
8746,815435,158241356,КХЛ,СКА,Авангард,1 минута. СЕРГЕЙ КОСТИЦЫН - 0:1!!! 10-й гол в ...,СКА – «Авангард» – 4:5 ОТ. Широков принес побе...,Сегодня в Петербурге проходит матч регулярного...
8747,1013451,1027691296,Кубок Гагарина,Барыс,Авангард,7 минута. гол! 1:0! Руденко! Получил шайбу в у...,«Барыс» – «Авангард» – 3:4. Курьянов забросил ...,Сегодня в четвертой игре серии первого раунда ...


In [ ]:
dt_frame['news_name_body'] = dt_frame['news_name'] + ' & ' + dt_frame['news_body']

In [ ]:
train_news_am, eval_news_am = train_test_split(dt_frame.sample(frac=1, random_state=42), test_size=1_000, random_state=42)

train_news_am, test_news_am = train_test_split(train_news_am, test_size=150, random_state=42)

In [ ]:
train_news_am.to_csv('train_after_match_pairs_data2.csv', index=False, encoding='utf-8')
eval_news_am.to_csv('eval_after_match_pairs_data2.csv', index=False, encoding='utf-8')
test_news_am.to_csv('test_after_match_pairs_data2.csv', index=False, encoding='utf-8')

!cp /content/train_after_match_pairs_data2.csv "/content/drive/MyDrive/RuSportSum/train_after_match_pairs_data2.csv"
!cp /content/eval_after_match_pairs_data2.csv "/content/drive/MyDrive/RuSportSum/eval_after_match_pairs_data2.csv"
!cp /content/test_after_match_pairs_data2.csv "/content/drive/MyDrive/RuSportSum/test_after_match_pairs_data2.csv"

#### Начало трансляции; реплики с каким-то типом (или вся трансляция, если нет типов); конец трансляции (НЕ ИСПОЛЬЗОВАЛОСЬ)

In [ ]:
pd.options.mode.chained_assignment = None

chosen_data =  pd.DataFrame()

for index in range(len(relevant_data_news)):
  current_news = relevant_data_news.iloc[index]

  # начало трансляции
  start_replicas = relevant_data_broadcasts[(relevant_data_broadcasts['match_id'] == current_news['match_id']) \
                                                & (relevant_data_broadcasts['type'].isin(content_type[2:]))]
  join_start_replicas = ' '.join(start_replicas.content.tolist())

  # все важные реплики из трансляции
  important_replicas = relevant_data_broadcasts[(relevant_data_broadcasts['match_id'] == current_news['match_id']) \
                                                & (relevant_data_broadcasts['type'].isin(content_type[2:]))]
  join_important_replicas = ' '.join(important_replicas.content.tolist())

  # концовка матча (все реплики, после реплики с каким-то типом)
  match_replicas = data_broadcasts[data_broadcasts['match_id'] == current_news['match_id']].reset_index()
  mask = match_replicas['type'].isin(content_type[1:]).to_list()
  try:
    index_true = mask[::-1].index(True)
  except:
    index_true = 1 if len(mask) > 1 else 0
  for i in range(len(mask) - index_true):
    mask[i] = True
  end_replicas = match_replicas[~pd.Series(mask)]
  join_end_match = ' '.join(end_replicas.content.tolist())

  # в матче нет важных реплик, поэтому возьмем все реплики матча
  if len(important_replicas) == 0:
    important_replicas = relevant_data_broadcasts[(relevant_data_broadcasts['match_id'] == current_news['match_id'])]

    important_replicas['id'] = current_news.loc['id']
    important_replicas['name_body'] = current_news.loc['name_body']

    chosen_data = pd.concat([chosen_data, important_replicas])
  else:
    start_replicas['id'] = current_news.loc['id']
    important_replicas['id'] = current_news.loc['id']
    end_replicas['id'] = current_news.loc['id']
    start_replicas['name_body'] = current_news.loc['name_body']
    important_replicas['name_body'] = current_news.loc['name_body']
    end_replicas['name_body'] = current_news.loc['name_body']

    chosen_data = pd.concat([chosen_data, start_replicas, important_replicas, end_replicas])

In [ ]:
columns = ['match_id', 'news_id', 'name_x', 'team1_x', 'team2_x', 'type', 'minute', 'content_y', 'news_name_body']

train_news_am = train_data.merge(chosen_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'id'])
train_news_am = train_news_am[columns]
train_news_am.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
train_news_am.insert(5, 'relevant_part', 0)

eval_news_am = eval_data.merge(chosen_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'id'])
eval_news_am = eval_news_am[columns]
eval_news_am.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
eval_news_am.insert(5, 'relevant_part', 0)

test_news_am = test_data.merge(chosen_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'id'])
test_news_am = test_news_am[columns]
test_news_am.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
test_news_am.insert(5, 'relevant_part', 0)

In [ ]:
train_news_am.to_csv('train_after_match_pairs_data3.csv', index=False, encoding='utf-8')
eval_news_am.to_csv('eval_after_match_pairs_data3.csv', index=False, encoding='utf-8')
test_news_am.to_csv('test_after_match_pairs_data3.csv', index=False, encoding='utf-8')

!cp /content/train_after_match_pairs_data3.csv "/content/drive/MyDrive/RuSportSum/train_after_match_pairs_data3.csv"
!cp /content/eval_after_match_pairs_data3.csv "/content/drive/MyDrive/RuSportSum/eval_after_match_pairs_data3.csv"
!cp /content/test_after_match_pairs_data3.csv "/content/drive/MyDrive/RuSportSum/test_after_match_pairs_data3.csv"

In [ ]:
cols = ['match_id', 'news_id']
temp_data = pd.concat([train_news_am, eval_news_am, test_news_am])

train_selector_am, eval_selector_am = train_test_split(pd.concat([train_data[cols], eval_data[cols], test_data[cols]]).drop_duplicates().reset_index(drop=True).sample(frac=1, random_state=3), test_size=100, random_state=3)
train_selector_am = train_selector_am.merge(temp_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'news_id'])
eval_selector_am = eval_selector_am.merge(temp_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'news_id'])

In [ ]:
# ['match_id'][495:500] =  [829986, 787160, 953621, 232581, 520095]
train_selector_am

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,ball,7,7 минута. Игуаин! гол! Четко щечкой положил мя...,Серхио Рамос одержал 150-ю победу с «Реалом» в...
1,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,ball,33,"33 минута. Ну, наконец-то! гол! Роналду! Убежа...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
2,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,ball,40,40 минута. БЕНЗЕМА! 3:0 обработал роскошный за...,Серхио Рамос одержал 150-ю победу с «Реалом» в...
3,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,ball,42,42 минута. Ха-ха! 3:1! Хаби Прието через рикош...,Серхио Рамос одержал 150-ю победу с «Реалом» в...
4,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,zamena,44,44 минута. Сурутуса вместо получившего поврежд...,Серхио Рамос одержал 150-ю победу с «Реалом» в...
...,...,...,...,...,...,...,...,...,...,...
254928,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. В последний раз так далеко словаки ...,Канада в третий раз подряд не сумела преодолет...
254929,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,"60 минута. Канада же в третий, повторяю, в тре...",Канада в третий раз подряд не сумела преодолет...
254930,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. Гецлафу из Хельсинки лучше лететь с...,Канада в третий раз подряд не сумела преодолет...
254931,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. А наша трансляция подошла к концу. ...,Канада в третий раз подряд не сумела преодолет...


In [ ]:
eval_selector_am

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,ball,17,17 минута. nan,«Амкар» – «Спартак». Гости впервые в своей ист...
1,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,yellowcard,20,"20 минута. Первое предупреждение в ""Спартаке"" ...",«Амкар» – «Спартак». Гости впервые в своей ист...
2,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,yellowcard,26,"26 минута. Первое предупреждение в ""Амкаре"" : ...",«Амкар» – «Спартак». Гости впервые в своей ист...
3,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,zamena,34,"34 минута. Первая замена в ""Амкаре"" : Роман Ге...",«Амкар» – «Спартак». Гости впервые в своей ист...
4,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,zamena,40,"40 минута. Вторая замена в ""Амкаре"" : Констант...",«Амкар» – «Спартак». Гости впервые в своей ист...
...,...,...,...,...,...,...,...,...,...,...
2980,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Кэйхилл напоследок в идеальном подк...,Англия – Италия – 1:2. Балотелли забил победны...
2981,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Сборная Италии стартует на чемпиона...,Англия – Италия – 1:2. Балотелли забил победны...
2982,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Итоговая статистика: удары (в створ...,Англия – Италия – 1:2. Балотелли забил победны...
2983,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,"90 минута. Назвать матч, который мы посмотрели...",Англия – Италия – 1:2. Балотелли забил победны...


In [ ]:
# пары для разметки и обучения селектора
train_selector_am.to_csv('train_selector_after_match1.csv', index=False, encoding='utf-8')
eval_selector_am.to_csv('eval_selector_after_match1.csv', index=False, encoding='utf-8')

!cp /content/train_selector_after_match1.csv "/content/drive/MyDrive/RuSportSum/train_selector_after_match1.csv"
!cp /content/eval_selector_after_match1.csv "/content/drive/MyDrive/RuSportSum/eval_selector_after_match1.csv"

#### Все реплики трансляции

In [ ]:
columns = ['match_id', 'news_id', 'name_x', 'team1_x', 'team2_x', 'type', 'minute', 'content_y', 'news_name_body']

train_news_am = train_data.merge(relevant_data_broadcasts, how='inner', left_on=['match_id'], right_on=['match_id'])
train_news_am = train_news_am[columns]
train_news_am.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
train_news_am.insert(5, 'relevant_part', 0)

eval_news_am = eval_data.merge(relevant_data_broadcasts, how='inner', left_on=['match_id'], right_on=['match_id'])
eval_news_am = eval_news_am[columns]
eval_news_am.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
eval_news_am.insert(5, 'relevant_part', 0)

test_news_am = test_data.merge(relevant_data_broadcasts, how='inner', left_on=['match_id'], right_on=['match_id'])
test_news_am = test_news_am[columns]
test_news_am.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
test_news_am.insert(5, 'relevant_part', 0)

In [ ]:
train_news_am.to_csv('train_after_match_pairs_data4.csv', index=False, encoding='utf-8')
eval_news_am.to_csv('eval_after_match_pairs_data4.csv', index=False, encoding='utf-8')
test_news_am.to_csv('test_after_match_pairs_data4.csv', index=False, encoding='utf-8')

!cp /content/train_after_match_pairs_data4.csv "/content/drive/MyDrive/RuSportSum/train_after_match_pairs_data4.csv"
!cp /content/eval_after_match_pairs_data4.csv "/content/drive/MyDrive/RuSportSum/eval_after_match_pairs_data4.csv"
!cp /content/test_after_match_pairs_data4.csv "/content/drive/MyDrive/RuSportSum/test_after_match_pairs_data4.csv"

In [ ]:
cols = ['match_id', 'news_id']
temp_data = pd.concat([train_news_am, eval_news_am, test_news_am])

train_selector_am, eval_selector_am = train_test_split(pd.concat([train_data[cols], eval_data[cols], test_data[cols]])[['match_id', 'news_id']].drop_duplicates().reset_index(drop=True).sample(frac=1, random_state=3), test_size=100, random_state=3)
train_selector_am = train_selector_am.merge(temp_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'news_id'])
eval_selector_am = eval_selector_am.merge(temp_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'news_id'])

train_selector_am.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)

In [ ]:
# ['match_id'][495:500] =  [829986, 787160, 953621, 232581, 520095]
train_selector_am

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,0,"0 минута. Салют амигос! ""Барса"" уже в трех очк...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
1,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,0,"0 минута. Без Пепе, без Озила и без Диарра. И ...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
2,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,1,1 минута. Поехали!,Серхио Рамос одержал 150-ю победу с «Реалом» в...
3,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,2,2 минута. Первый удар за гостями - вовремя под...,Серхио Рамос одержал 150-ю победу с «Реалом» в...
4,383401,139069154,Испания. Ла Лига,Реал Мадрид,Реал Сосьедад,0,NaN,5,"5 минута. ""Реал"" начал раскачивать. А Бензема ...",Серхио Рамос одержал 150-ю победу с «Реалом» в...
...,...,...,...,...,...,...,...,...,...,...
748696,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. В последний раз так далеко словаки ...,Канада в третий раз подряд не сумела преодолет...
748697,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,"60 минута. Канада же в третий, повторяю, в тре...",Канада в третий раз подряд не сумела преодолет...
748698,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. Гецлафу из Хельсинки лучше лететь с...,Канада в третий раз подряд не сумела преодолет...
748699,475786,140849366,Чемпионат мира,Канада,Словакия,0,empty,60,60 минута. А наша трансляция подошла к концу. ...,Канада в третий раз подряд не сумела преодолет...


In [ ]:
eval_selector_am

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,empty,0,0 минута. nan,«Амкар» – «Спартак». Гости впервые в своей ист...
1,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,empty,0,"0 минута. Добрый вечер, друзья. Добро пожалова...",«Амкар» – «Спартак». Гости впервые в своей ист...
2,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,empty,0,"0 минута. На данный момент ""Амкар"" занимает в ...",«Амкар» – «Спартак». Гости впервые в своей ист...
3,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,empty,0,"0 минута. К этой игре ""Амкар"" подошел с серией...",«Амкар» – «Спартак». Гости впервые в своей ист...
4,787025,152386849,Россия. Премьер-лига,Амкар,Спартак,0,empty,0,"0 минута. ""Спартак"" в этом чемпионате вообще н...",«Амкар» – «Спартак». Гости впервые в своей ист...
...,...,...,...,...,...,...,...,...,...,...
8755,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Кэйхилл напоследок в идеальном подк...,Англия – Италия – 1:2. Балотелли забил победны...
8756,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Сборная Италии стартует на чемпиона...,Англия – Италия – 1:2. Балотелли забил победны...
8757,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,90 минута. Итоговая статистика: удары (в створ...,Англия – Италия – 1:2. Балотелли забил победны...
8758,829996,1021343013,Чемпионат мира,Англия,Италия,0,empty,90,"90 минута. Назвать матч, который мы посмотрели...",Англия – Италия – 1:2. Балотелли забил победны...


In [ ]:
# пары для разметки и обучения селектора
train_selector_am.to_csv('train_selector_after_match2.csv', index=False, encoding='utf-8')
eval_selector_am.to_csv('eval_selector_after_match2.csv', index=False, encoding='utf-8')

!cp /content/train_selector_after_match2.csv "/content/drive/MyDrive/RuSportSum/train_selector_after_match2.csv"
!cp /content/eval_selector_after_match2.csv "/content/drive/MyDrive/RuSportSum/eval_selector_after_match2.csv"

## Новости, посвященные какому-то событию

In [ ]:
during_match_data = result_data[result_data['result_match'] == 0]

relevant_match_ids = during_match_data['match_id'].drop_duplicates().tolist()

relevant_data_broadcasts = data_broadcasts[data_broadcasts['match_id'].isin(relevant_match_ids)].reset_index(drop=True)
relevant_data_news = during_match_data

In [ ]:
relevant_data_broadcasts

,match_id,team1,team2,name,match_time,type,minute,content,message_time
0,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,0 минута. Добрый день! Sports.ru поздравляет в...,1237034763
1,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. Будем надеяться, что признаков пресл...",1237034996
2,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. Что ж, сегодняшних соперников можно ...",1237035202
3,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,0 минута. Зимой подмосковный клуб едва не исче...,1237035415
4,18925,Локомотив,Химки,Россия. Премьер-лига,2009-03-14 13:30:00,NaN,0,"0 минута. ""Локомотив"" зимой старался стать луч...",1237035796
...,...,...,...,...,...,...,...,...,...
205899,1104865,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-05-31 00:00:00,empty,60,"60 минута. Первый раунд противостояния за ""пин...",1464664182
205900,1104865,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-05-31 00:00:00,empty,60,"60 минута. Бонино, Шери и Марло названы лучшим...",1464664271
205901,1104865,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-05-31 00:00:00,empty,60,"60 минута. На мой взгляд, место Марло в тройке...",1464664338
205902,1104865,Питтсбург,Сан-Хосе,Кубок Стэнли,2016-05-31 00:00:00,empty,60,60 минута. Евгений Малкин сыграл не самый свой...,1464664439


In [ ]:
count_news = len(relevant_data_news)
print("Количество новостей:", count_news)

Количество новостей: 4089


#### Только реплики, которые содержат имена из новости

In [ ]:
df = {}
count_df_row = 0
stemmer = SnowballStemmer("russian")
russian_stopwords = stopwords.words("russian")

for index in range(len(relevant_data_news)):
  current_news = relevant_data_news.iloc[index]
  team1 = current_news['team1'].lower()
  team2 = current_news['team2'].lower()

  names = re.findall(r"[А-Я'][а-я]+", current_news['name_body'])
  clear_names = []
  for name in names:
    name = name.lower()
    if name not in russian_stopwords and name not in team1 and name not in team2 and name not in clear_names:
      clear_names.append(name)

  contents = relevant_data_broadcasts[relevant_data_broadcasts['match_id'] == current_news['match_id']]

  important_replicas = contents[contents.content.apply(lambda row: any(name in row.lower() for name in clear_names))]
  join_important_replicas = ' '.join(important_replicas.content.tolist())

  df['row_' + str(count_df_row)] = [
      current_news['match_id'],
      current_news['id'],
      current_news['broadcast_name'],
      current_news['team1'],
      current_news['team2'],
      join_important_replicas,
      current_news['news_name'],
      current_news['body']
      ]
  count_df_row += 1

In [ ]:
dt_frame = pd.DataFrame.from_dict(df, orient='index', columns=['match_id', 'news_id', 'name', 'team1', 'team2', 'content', 'news_name', 'news_body']).reset_index(drop=True)
dt_frame

,match_id,news_id,name,team1,team2,content,news_name,news_body
0,476651,140901107,Лига чемпионов,Бавария,Челси,"0 минута. Тем более, что поводов для оной пред...",Дрогба сравнял счет в матче с «Баварией»,Форвард «Челси» Дидье Дрогба забил гол в финал...
1,455470,141722676,Чемпионат Европы,Германия,Италия,0 минута. Sports.ru приглашает вас присоединит...,Балотелли сделал дубль в матче против Германии,Форвард сборной Италии Марио Балотелли забил г...
2,1028663,1030654794,Кубок Америки,Бразилия,Колумбия,"0 минута. Да-да, колумбийцы не просо входят в ...",Кубок Америки-2015. Неймар дисквалифицирован н...,Нападающий сборной Бразилии Неймар пропустит д...
3,222661,73243491,Англия. Премьер-лига,Арсенал,Вест Бромвич,"0 минута. Всем привет. С учетом того, что толь...",Одемвинги забил гол в ворота «Арсенала»,Нападающий «Вест Бромвича» Питер Одемвинги отл...
4,479929,152692472,Квалификация ЧМ: Европа,Россия,Люксембург,"0 минута. Сегодня в Казани - городе, самого гр...",Кокорин забил самый быстрый гол в истории сбор...,Нападающий московского «Динамо» и сборной Росс...
...,...,...,...,...,...,...,...,...
4084,801785,151903293,Лига чемпионов,Зенит,Нордшелланд,0 минута. 15 раз российские клубы в еврокубках...,«Зенит» – «Нордшелланд». Нету удален с поля,Защитник «Зенита» Луиш Нету получил красную ка...
4085,955773,1027615975,Англия. Премьер-лига,Ливерпуль,Манчестер Сити,"0 минута. Например, удержать планку голов в во...",«Ливерпуль» – «Манчестер Сити». Джеко забил 50...,Форвард «Манчестер Сити» Эдин Джеко сравнял сч...
4086,830040,1022114821,Чемпионат мира,Бразилия,Голландия,0 минута. Но у этого финала есть особый подтек...,Бразилия – Голландия. Снейдер получил травму п...,Полузащитник сборной Голландии Уэсли Снейдер н...
4087,799065,154776181,Испания. Ла Лига,Атлетик,Барселона,25 минута. Монтойя! На скорости получил мяч в ...,«Атлетик» – «Барселона». Пике проводит 150-й м...,Защитник «Барселоны» Жерар Пике вышел в старто...


In [ ]:
dt_frame['news_name_body'] = dt_frame['news_name'] + ' & ' + dt_frame['news_body']

In [ ]:
train_news_dm, eval_news_dm = train_test_split(dt_frame.sample(frac=1, random_state=42), test_size=500, random_state=42)

train_news_dm, test_news_dm = train_test_split(train_news_dm, test_size=150, random_state=42)

In [ ]:
train_news_dm

,match_id,news_id,name,team1,team2,content,news_name,news_body,news_name_body
2436,479927,149901417,Квалификация ЧМ: Европа,Португалия,Россия,0 минута. Всем привет! Непобедимая сборная Рос...,Португалия – Россия. Акинфеев установил рекорд...,Форвард сборной Португалии Элдер Поштига откры...,Португалия – Россия. Акинфеев установил рекорд...
39,286145,129843619,Россия. Премьер-лига,Ростов,Зенит,"0 минута. Здравствуйте, друзья. 28-й тур чемпи...",Плетикоса получил травму в матче с «Зенитом»,Вратарь «Ростова» Стипе Плетикоса получил трав...,Плетикоса получил травму в матче с «Зенитом» &...
172,1095102,1038508209,Товарищеские матчи (сборные),Германия,Англия,0 минута. Всем привет! Можно сколько угодно не...,Германия – Англия. Кейн перед голом повторил п...,Форвард сборной Англии Харри Кейн отыграл один...,Германия – Англия. Кейн перед голом повторил п...
216,471129,137844886,Кубок Гагарина,ЦСКА,СКА,10 минута. Федоров у борта в чужой зоне впечат...,Прохоркин набрал первое очко в плей-офф,Нападающий ЦСКА Николай Прохоркин в четвертом ...,Прохоркин набрал первое очко в плей-офф & Напа...
272,455471,141780495,Чемпионат Европы,Испания,Италия,0 минута. Ради такого дела Дель Боске снова ре...,Кьеллини получил травму бедра в финале Евро-2012,Защитник «Ювентса» и сборной Италии Джорджо Кь...,Кьеллини получил травму бедра в финале Евро-20...
...,...,...,...,...,...,...,...,...,...
3380,306385,129852656,Англия. Премьер-лига,Манчестер Юнайтед,Манчестер Сити,35 минута. Сильва слишком сильно подал стандар...,Эванс получил красную карточку за фол на Балот...,Защитник «Манчестер Юнайтед» Джонни Эванс был ...,Эванс получил красную карточку за фол на Балот...
2569,296562,96016458,Чемпионат мира,Чехия,Россия,"0 минута. Ну что, попробуем помечтать? Илья Ко...",Артюхин разбил голову Рахунеку,Защитник сборной Чехии Карел Рахунек получил т...,Артюхин разбил голову Рахунеку & Защитник сбор...
2381,1047368,1035867637,Испания. Ла Лига,Барселона,Бетис,0 минута. В 2015 «Барса» взяла почти все возмо...,«Барселона» – «Бетис». Месси проводит 500-й ма...,Нападающий «Барселоны» Лионель Месси вышел в с...,«Барселона» – «Бетис». Месси проводит 500-й ма...
633,955670,1025858505,Англия. Премьер-лига,Астон Вилла,Манчестер Юнайтед,7 минута. Окоре здорово прорвался по флангу. А...,«Астон Вилла» – «МЮ». Агбонлахор удален с поля,Форвард «Астон Виллы» Габриэль Агбонлахор был ...,«Астон Вилла» – «МЮ». Агбонлахор удален с поля...


In [ ]:
train_news_dm.to_csv('train_during_match_pairs_data1.csv', index=False, encoding='utf-8')
eval_news_dm.to_csv('eval_during_match_pairs_data1.csv', index=False, encoding='utf-8')
test_news_dm.to_csv('test_during_match_pairs_data1.csv', index=False, encoding='utf-8')

!cp /content/train_during_match_pairs_data1.csv "/content/drive/MyDrive/RuSportSum/train_during_match_pairs_data1.csv"
!cp /content/eval_during_match_pairs_data1.csv "/content/drive/MyDrive/RuSportSum/eval_during_match_pairs_data1.csv"
!cp /content/test_during_match_pairs_data1.csv "/content/drive/MyDrive/RuSportSum/test_during_match_pairs_data1.csv"

In [ ]:
# данные для сверки, что везде одинаковые наборы
train_during_data = train_news_dm
eval_during_data = eval_news_dm
test_during_data = test_news_dm

#### Все реплики трансляции

In [ ]:
columns = ['match_id', 'news_id', 'name_x', 'team1_x', 'team2_x', 'type', 'minute', 'content_y', 'news_name_body']

train_news_dm = train_during_data.merge(relevant_data_broadcasts, how='inner', left_on=['match_id'], right_on=['match_id'])
train_news_dm = train_news_dm[columns]
train_news_dm.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
train_news_dm.insert(5, 'relevant_part', 0)

eval_news_dm = eval_during_data.merge(relevant_data_broadcasts, how='inner', left_on=['match_id'], right_on=['match_id'])
eval_news_dm = eval_news_dm[columns]
eval_news_dm.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
eval_news_dm.insert(5, 'relevant_part', 0)

test_news_dm = test_during_data.merge(relevant_data_broadcasts, how='inner', left_on=['match_id'], right_on=['match_id'])
test_news_dm = test_news_dm[columns]
test_news_dm.rename(columns={"name_x": "name", "team1_x": "team1", "team2_x": "team2", "content_y": "content"}, inplace=True)
test_news_dm.insert(5, 'relevant_part', 0)

In [ ]:
train_news_dm.to_csv('train_during_match_pairs_data3.csv', index=False, encoding='utf-8')
eval_news_dm.to_csv('eval_during_match_pairs_data3.csv', index=False, encoding='utf-8')
test_news_dm.to_csv('test_during_match_pairs_data3.csv', index=False, encoding='utf-8')

!cp /content/train_during_match_pairs_data3.csv "/content/drive/MyDrive/RuSportSum/train_during_match_pairs_data3.csv"
!cp /content/eval_during_match_pairs_data3.csv "/content/drive/MyDrive/RuSportSum/eval_during_match_pairs_data3.csv"
!cp /content/test_during_match_pairs_data3.csv "/content/drive/MyDrive/RuSportSum/test_during_match_pairs_data3.csv"

In [ ]:
cols = ['match_id', 'news_id']
temp_data = pd.concat([train_news_dm, eval_news_dm, test_news_dm])

train_selector_dm, eval_selector_dm = train_test_split(pd.concat([train_during_data[cols], eval_during_data[cols], test_during_data[cols]]).drop_duplicates().reset_index(drop=True).sample(frac=1, random_state=3), test_size=100, random_state=3)
train_selector_dm = train_selector_dm.merge(temp_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'news_id'])
eval_selector_dm = eval_selector_dm.merge(temp_data, how='inner', left_on=['match_id', 'news_id'], right_on=['match_id', 'news_id'])

In [ ]:
# ['match_id'][495:500] =  [787061, 980343, 32419, 980293, 290315]
train_selector_dm

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,0 минута. Приветствуем Всех любителей самой по...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
1,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,0 минута. Предлагаем вашему вниманию прямую те...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
2,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,0 минута. Ростовчане весьма уверенно выступают...,«Ростов» – «Амкар». Вавжиняк удален с поля & З...
3,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,"0 минута. Более того, будь ""Ростов"" чуть удачл...",«Ростов» – «Амкар». Вавжиняк удален с поля & З...
4,787156,158820706,Россия. Премьер-лига,Ростов,Амкар,0,empty,0,"0 минута. ""Амкар"", подумать только, всего на 3...",«Ростов» – «Амкар». Вавжиняк удален с поля & З...
...,...,...,...,...,...,...,...,...,...,...
328467,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,95,95 минута. Сейчас пойдет компенсированное врем...,«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328468,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,attention,96,"96 минута. Все! 1:2 - впервые за 9 лет ""Арсена...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328469,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,96,"96 минута. Руни снова забил ""канонирам"", но ег...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...
328470,1013052,1027838659,Англия. Кубок,Манчестер Юнайтед,Арсенал,0,empty,97,"97 минута. Спасибо, что были с нами! Завтра от...",«МЮ» – «Арсенал». Окслейд-Чемберлен получил тр...


In [ ]:
eval_selector_dm

,match_id,news_id,name,team1,team2,relevant_part,type,minute,content,news_name_body
0,1095117,1038599413,Товарищеские матчи (сборные),Франция,Россия,0,empty,0,0 минута. Пермете муа де ме презенте – же ма п...,Франция – Россия. Эвра и Матье получили травмы...
1,1095117,1038599413,Товарищеские матчи (сборные),Франция,Россия,0,empty,0,0 минута. 17 лет назад состоялся лучший матч н...,Франция – Россия. Эвра и Матье получили травмы...
2,1095117,1038599413,Товарищеские матчи (сборные),Франция,Россия,0,empty,0,0 минута.,Франция – Россия. Эвра и Матье получили травмы...
3,1095117,1038599413,Товарищеские матчи (сборные),Франция,Россия,0,empty,0,0 минута. Очевидно одно – сегодняшний состав с...,Франция – Россия. Эвра и Матье получили травмы...
4,1095117,1038599413,Товарищеские матчи (сборные),Франция,Россия,0,empty,0,"0 минута. С другой стороны, у нас так: проигры...",Франция – Россия. Эвра и Матье получили травмы...
...,...,...,...,...,...,...,...,...,...,...
8247,787013,151814813,Россия. Премьер-лига,Рубин,ЦСКА,0,empty,90,90 минута. Вторая безголевая ничья в нынешнем ...,«Рубин» – ЦСКА. Карадениз получил травму & Пол...
8248,787013,151814813,Россия. Премьер-лига,Рубин,ЦСКА,0,empty,90,"90 минута. В целом, матч не был скучным, а во ...",«Рубин» – ЦСКА. Карадениз получил травму & Пол...
8249,787013,151814813,Россия. Премьер-лига,Рубин,ЦСКА,0,empty,90,90 минута. Ансальди совершает круг почета по с...,«Рубин» – ЦСКА. Карадениз получил травму & Пол...
8250,787013,151814813,Россия. Премьер-лига,Рубин,ЦСКА,0,empty,90,90 минута. Статистика матча. По владению мячом...,«Рубин» – ЦСКА. Карадениз получил травму & Пол...


In [ ]:
# пары для разметки и обучения селектора
train_selector_dm.to_csv('train_selector_during_match2.csv', index=False, encoding='utf-8')
eval_selector_dm.to_csv('eval_selector_during_match2.csv', index=False, encoding='utf-8')

!cp /content/train_selector_during_match2.csv "/content/drive/MyDrive/RuSportSum/train_selector_during_match2.csv"
!cp /content/eval_selector_during_match2.csv "/content/drive/MyDrive/RuSportSum/eval_selector_during_match2.csv"